In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [ ]:
#################################################
# You can change this cell to update data list
train_lst = ['data_track1_1.csv', 'data_track1_2.csv', 'data_track2_1.csv', 'data_track2_2.csv']
val_lst = ['data_track1_val.csv', 'data_track2_val.csv']

In [ ]:
#################################################
# DO NOT CHANGE THIS CELL(unless you have a better idea to preprocess the data)
train_data = np.loadtxt(train_lst[0], delimiter = ",")
print(train_lst[0], ":", train_data.shape)
for i in range(1, len(train_lst)):
    new_data = np.loadtxt(train_lst[i], delimiter = ",")
    print(train_lst[i], ":", new_data.shape)
    train_data = np.concatenate((train_data, new_data), axis=0)
num_train = train_data.shape[0]

val_data = np.loadtxt(val_lst[0], delimiter = ",")
print(val_lst[0], ":", val_data.shape)
for i in range(1, len(val_lst)):
    new_data = np.loadtxt(val_lst[i], delimiter = ",")
    print(val_lst[i], ":", new_data.shape)
    val_data = np.concatenate((val_data, new_data), axis=0)
num_val = val_data.shape[0]

data = np.concatenate((train_data, val_data), axis=0)
data = torch.from_numpy(data).float()

processed_data = torch.nan_to_num(data, posinf=11.0)
means = torch.mean(processed_data, 0)
stds = torch.std(processed_data, 0)
norm_data = (processed_data - means) / stds

data_track1_1.csv : (3340, 1083)
data_track1_2.csv : (3340, 1083)
data_track2_1.csv : (822, 1083)
data_track2_2.csv : (826, 1083)
data_track1_val.csv : (334, 1083)
data_track2_val.csv : (83, 1083)


In [ ]:
#################################################
# You can change this cell to update data list
input_dim = 1081
hidden_dim1 = 1000
hidden_dim2 = 500

X_train = norm_data[:num_train, :-2]
X_val = norm_data[num_train:, :-2]
y_train = norm_data[:num_train, -1]
y_val = norm_data[num_train:, -1]
print("X_train:", X_train.shape)
print("X_val:", X_val.shape)
print("y_train:", y_train.shape)
print("y_val:", y_val.shape)

X_train: torch.Size([8328, 1081])
X_val: torch.Size([417, 1081])
y_train: torch.Size([8328])
y_val: torch.Size([417])


In [ ]:
#################################################
# You can change this cell to make variations to model
class Model(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, 1)
    def forward(self, x):
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [ ]:
model = Model(input_dim, hidden_dim1, hidden_dim2)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0005)

for epoch in range(50):
    loss_train = 0
    for i in range(X_train.shape[0]):
        X = X_train[i, :]
        y = y_train[i]
        s = model(X)
        loss = criterion(s, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_train = loss_train + loss

    loss_val = 0
    for j in range(X_val.shape[0]):
        X = X_val[j, :]
        y = y_val[j]
        s = model(X)
        loss_val = loss_val + criterion(s, y)
    print(epoch, ":", loss_train / X_train.shape[0], loss_val / X_val.shape[0])

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0 : tensor(0.3214, grad_fn=<DivBackward0>) tensor(0.5859, grad_fn=<DivBackward0>)
1 : tensor(0.2246, grad_fn=<DivBackward0>) tensor(0.3798, grad_fn=<DivBackward0>)
2 : tensor(0.1886, grad_fn=<DivBackward0>) tensor(0.3202, grad_fn=<DivBackward0>)
3 : tensor(0.1703, grad_fn=<DivBackward0>) tensor(0.2963, grad_fn=<DivBackward0>)
4 : tensor(0.1588, grad_fn=<DivBackward0>) tensor(0.2804, grad_fn=<DivBackward0>)
5 : tensor(0.1500, grad_fn=<DivBackward0>) tensor(0.2620, grad_fn=<DivBackward0>)
6 : tensor(0.1428, grad_fn=<DivBackward0>) tensor(0.2460, grad_fn=<DivBackward0>)
7 : tensor(0.1370, grad_fn=<DivBackward0>) tensor(0.2320, grad_fn=<DivBackward0>)
8 : tensor(0.1320, grad_fn=<DivBackward0>) tensor(0.2206, grad_fn=<DivBackward0>)
9 : tensor(0.1276, grad_fn=<DivBackward0>) tensor(0.2095, grad_fn=<DivBackward0>)
10 : tensor(0.1238, grad_fn=<DivBackward0>) tensor(0.2042, grad_fn=<DivBackward0>)
11 : tensor(0.1203, grad_fn=<DivBackward0>) tensor(0.1995, grad_fn=<DivBackward0>)
12 : tensor(0.

In [ ]:
torch.save(model, "/content/model.pt")
torch.save(means, "/content/means.pt")
torch.save(stds, "/content/stds.pt")